## This notebook will be mainly used for the Segmenting and Clustering Neighborhoods in Toronto.

### Initial Requirements

In [ ]:
#!conda install lxml --y

In [99]:
#!conda install openpyxl --y

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.2

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - openpyxl


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openpyxl-3.0.3             |             py_0         158 KB
    et_xmlfile-1.0.1           |           py36_0          20 KB
    openssl-1.1.1d             |       h7b6447c_4         3.7 MB
    certifi-2019.11.28         |           py36_0         156 KB
    jdcal-1.4.1                |             py_0          11 KB
    ------------------------------------------------------------
                                           Total:         4.1 MB

The following packages will be UPDATED:

    certifi:    2019.9.11-p

In [ ]:
pip install lxml

In [ ]:
pip install beautifulsoup4

In [75]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
from pandas.io.html import read_html
import numpy as np # library to handle data in a vectorized manner
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
from sklearn import preprocessing

from sklearn.cluster import KMeans
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

## Getting the table

In [2]:
airdf = pd.read_csv("AB_NYC_2019.csv")

In [3]:
airdf.shape

(48895, 16)

In [4]:
airdf

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48890,36484665,Charming one bedroom - newly renovated rowhouse,8232441,Sabrina,Brooklyn,Bedford-Stuyvesant,40.67853,-73.94995,Private room,70,2,0,NaN,NaN,2,9
48891,36485057,Affordable room in Bushwick/East Williamsburg,6570630,Marisol,Brooklyn,Bushwick,40.70184,-73.93317,Private room,40,4,0,NaN,NaN,2,36
48892,36485431,Sunny Studio at Historical Neighborhood,23492952,Ilgar & Aysel,Manhattan,Harlem,40.81475,-73.94867,Entire home/apt,115,10,0,NaN,NaN,1,27
48893,36485609,43rd St. Time Square-cozy single bed,30985759,Taz,Manhattan,Hell's Kitchen,40.75751,-73.99112,Shared room,55,1,0,NaN,NaN,6,2


### Dropping unnecesary columns

In [5]:
airdf.drop(columns=['id', 'host_name', 'neighbourhood', 'name', 'host_id', 'room_type', 'number_of_reviews', 'last_review', 'reviews_per_month', 'calculated_host_listings_count'], inplace=True)

In [6]:
airdf

,neighbourhood_group,latitude,longitude,price,minimum_nights,availability_365
0,Brooklyn,40.64749,-73.97237,149,1,365
1,Manhattan,40.75362,-73.98377,225,1,355
2,Manhattan,40.80902,-73.94190,150,3,365
3,Brooklyn,40.68514,-73.95976,89,1,194
4,Manhattan,40.79851,-73.94399,80,10,0
...,...,...,...,...,...,...
48890,Brooklyn,40.67853,-73.94995,70,2,9
48891,Brooklyn,40.70184,-73.93317,40,4,36
48892,Manhattan,40.81475,-73.94867,115,10,27
48893,Manhattan,40.75751,-73.99112,55,1,2


#### We don't know how many nights we are going to stay so we will filter the minimum nights to 1

In [7]:
# Get names of indexes for which column minimum_nights has value > 1
indexNames = airdf[airdf['minimum_nights'] > 1 ].index 
# Delete these row indexes from dataFrame
airdf.drop(indexNames , inplace=True)

In [8]:
airdf

,neighbourhood_group,latitude,longitude,price,minimum_nights,availability_365
0,Brooklyn,40.64749,-73.97237,149,1,365
1,Manhattan,40.75362,-73.98377,225,1,355
3,Brooklyn,40.68514,-73.95976,89,1,194
9,Manhattan,40.71344,-73.99037,150,1,188
17,Manhattan,40.74192,-73.99501,140,1,12
...,...,...,...,...,...,...
48878,Queens,40.74477,-73.87727,45,1,172
48880,Brooklyn,40.71232,-73.94220,120,1,365
48887,Brooklyn,40.71232,-73.94220,170,1,365
48889,Queens,40.69137,-73.80844,65,1,163


In [9]:
#now we can drop the minimum_nights
airdf.drop(columns=['minimum_nights'], inplace=True)

#### We want to filter the availability at least of half a year

In [10]:
# Get names of indexes for which column availability has value < 183
indexNames = airdf[airdf['availability_365'] < 183 ].index 
# Delete these row indexes from dataFrame
airdf.drop(indexNames , inplace=True)

In [11]:
airdf

,neighbourhood_group,latitude,longitude,price,availability_365
0,Brooklyn,40.64749,-73.97237,149,365
1,Manhattan,40.75362,-73.98377,225,355
3,Brooklyn,40.68514,-73.95976,89,194
9,Manhattan,40.71344,-73.99037,150,188
22,Brooklyn,40.67989,-73.97798,80,364
...,...,...,...,...,...
48851,Queens,40.69137,-73.86591,35,341
48867,Brooklyn,40.59080,-73.97116,25,338
48873,Brooklyn,40.63116,-73.92616,170,363
48880,Brooklyn,40.71232,-73.94220,120,365


In [12]:
#now we can drop the availability_365
airdf.drop(columns=['availability_365'], inplace=True)

## Reading a criminality dataset

#### Just because we want to have a good time we will bear in mind some criminality data associated to neighbourhoods

In [13]:
crime = pd.read_csv("NYPD_Complaint_Data_Current_YTD.csv")

In [14]:
crime

,CMPLNT_NUM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,RPT_DT,KY_CD,OFNS_DESC,PD_CD,PD_DESC,...,ADDR_PCT_CD,LOC_OF_OCCUR_DESC,PREM_TYP_DESC,PARKS_NM,HADEVELOPT,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lat_Lon
0,736216184,09/30/2016,23:25:00,09/30/2016,23:25:00,09/30/2016,236,DANGEROUS WEAPONS,782.0,"WEAPONS, POSSESSION, ETC",...,42,NaN,TRANSIT - NYC SUBWAY,NaN,NaN,1015308.0,244373.0,40.837376,-73.887761,"(40.837376359, -73.887760929)"
1,294332956,09/30/2016,23:16:00,09/30/2016,23:21:00,09/30/2016,344,ASSAULT 3 & RELATED OFFENSES,101.0,ASSAULT 3,...,71,OPPOSITE OF,STREET,NaN,NaN,997932.0,180172.0,40.661205,-73.950687,"(40.661204871, -73.950686652)"
2,852981427,09/30/2016,23:00:00,09/30/2016,23:05:00,09/30/2016,235,DANGEROUS DRUGS,567.0,"MARIJUANA, POSSESSION 4 & 5",...,43,INSIDE,RESIDENCE - PUBLIC HOUSING,NaN,CASTLE HILL,1025580.0,236918.0,40.816872,-73.850685,"(40.816872438, -73.850684927)"
3,369976063,09/30/2016,23:00:00,NaN,NaN,09/30/2016,118,DANGEROUS WEAPONS,793.0,WEAPONS POSSESSION 3,...,103,NaN,STREET,NaN,NaN,1038464.0,192970.0,40.696177,-73.804492,"(40.696177006, -73.804492266)"
4,117213771,09/30/2016,23:00:00,09/30/2016,23:10:00,09/30/2016,578,HARRASSMENT 2,637.0,"HARASSMENT,SUBD 1,CIVILIAN",...,110,FRONT OF,STREET,NaN,NaN,1016301.0,209428.0,40.741458,-73.884339,"(40.741458245, -73.884339073)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24281,317497826,09/12/2016,19:40:00,09/12/2016,19:43:00,09/12/2016,341,PETIT LARCENY,343.0,"LARCENY,PETIT OF BICYCLE",...,67,NaN,PARK/PLAYGROUND,NaN,NaN,998608.0,171942.0,40.638614,-73.948268,"(40.638614237, -73.948267625)"
24282,827964069,09/12/2016,19:35:00,09/12/2016,19:40:00,09/12/2016,235,DANGEROUS DRUGS,567.0,"MARIJUANA, POSSESSION 4 & 5",...,30,FRONT OF,STREET,NaN,NaN,996889.0,239161.0,40.823116,-73.954335,"(40.823116479, -73.954335162)"
24283,585540160,09/12/2016,19:35:00,09/12/2016,19:40:00,09/12/2016,348,VEHICLE AND TRAFFIC LAWS,916.0,LEAVING SCENE-ACCIDENT-PERSONA,...,42,FRONT OF,STREET,NaN,BUTLER,1010169.0,244577.0,40.837953,-73.906332,"(40.837952857, -73.90633196)"
24284,924661366,09/12/2016,19:30:00,09/12/2016,19:45:00,09/12/2016,235,DANGEROUS DRUGS,567.0,"MARIJUANA, POSSESSION 4 & 5",...,6,FRONT OF,PARK/PLAYGROUND,NaN,NaN,985305.0,205004.0,40.729373,-73.996195,"(40.729373465, -73.996194963)"


#### Not going for the details, we just want the neighbourhoods

In [19]:
crime_neighbourhoods = pd.DataFrame(data=crime['BORO_NM'])

In [20]:
crime_neighbourhoods

,BORO_NM
0,BRONX
1,BROOKLYN
2,BRONX
3,QUEENS
4,QUEENS
...,...
24281,BROOKLYN
24282,MANHATTAN
24283,BRONX
24284,MANHATTAN


### We must quantify the crime intensity

In [29]:
crime_intensity = pd.DataFrame(data=crime_neighbourhoods.groupby('BORO_NM').size())

In [30]:
crime_intensity

,0
BORO_NM,
BRONX,5289
BROOKLYN,7345
MANHATTAN,5780
QUEENS,4736
STATEN ISLAND,1136


### And it would be great to normalize it to 1

In [32]:
x = crime_intensity.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
crime_intensity_norm = pd.DataFrame(x_scaled)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [33]:
crime_intensity_norm

,0
0,0.668868
1,1.000000
2,0.747947
3,0.579804
4,0.000000


### So we can substitute the actual normalized crime intensity into the previous df

In [53]:
for i in range(0,5):
    crime_intensity[0][i] = crime_intensity_norm[0][i]

In [56]:
crime_intensity

,0
BORO_NM,
BRONX,0.668868
BROOKLYN,1.000000
MANHATTAN,0.747947
QUEENS,0.579804
STATEN ISLAND,0.000000


### We do also have to convert the index into the neighbourhood column being adapted to the other table

In [57]:
crime_intensity['neighbourhood_group'] = crime_intensity.index

In [60]:
neighbourhoodsadapted = ['Bronx', 'Brooklyn', 'Manhattan', 'Queens', 'Staten island']
for i in range(0,5):
    crime_intensity['neighbourhood_group'][i] = neighbourhoodsadapted[i]

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [61]:
crime_intensity

,0,neighbourhood_group
BORO_NM,,
BRONX,0.668868,Bronx
BROOKLYN,1.000000,Brooklyn
MANHATTAN,0.747947,Manhattan
QUEENS,0.579804,Queens
STATEN ISLAND,0.000000,Staten island


### Now we can merge the dfs

In [331]:
merged_airdf = airdf.merge(crime_intensity, on=('neighbourhood_group'))

In [332]:
merged_airdf

,neighbourhood_group,latitude,longitude,price,0
0,Brooklyn,40.64749,-73.97237,149,1.000000
1,Brooklyn,40.68514,-73.95976,89,1.000000
2,Brooklyn,40.67989,-73.97798,80,1.000000
3,Brooklyn,40.68310,-73.95473,70,1.000000
4,Brooklyn,40.63702,-73.96327,150,1.000000
...,...,...,...,...,...
3490,Queens,40.75104,-73.81459,45,0.579804
3491,Queens,40.73710,-73.88086,50,0.579804
3492,Queens,40.69185,-73.86431,37,0.579804
3493,Queens,40.69183,-73.86523,34,0.579804


In [333]:
merged_airdf.rename(columns={0:'criminality_index'}, inplace=True)

In [334]:
merged_airdf

,neighbourhood_group,latitude,longitude,price,criminality_index
0,Brooklyn,40.64749,-73.97237,149,1.000000
1,Brooklyn,40.68514,-73.95976,89,1.000000
2,Brooklyn,40.67989,-73.97798,80,1.000000
3,Brooklyn,40.68310,-73.95473,70,1.000000
4,Brooklyn,40.63702,-73.96327,150,1.000000
...,...,...,...,...,...
3490,Queens,40.75104,-73.81459,45,0.579804
3491,Queens,40.73710,-73.88086,50,0.579804
3492,Queens,40.69185,-73.86431,37,0.579804
3493,Queens,40.69183,-73.86523,34,0.579804


## Define cluster areas k=10

In [335]:
# set number of clusters
kclusters = 10
merged_airdf_clustering = merged_airdf.drop(columns=['neighbourhood_group', 'latitude' , 'longitude'])

In [336]:
merged_airdf_clustering

,price,criminality_index
0,149,1.000000
1,89,1.000000
2,80,1.000000
3,70,1.000000
4,150,1.000000
...,...,...
3490,45,0.579804
3491,50,0.579804
3492,37,0.579804
3493,34,0.579804


In [339]:
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(merged_airdf_clustering)
# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 
# add clustering labels
merged_airdf.insert(0, 'Cluster_Labels', kmeans.labels_)

In [340]:
merged_airdf

,Cluster_Labels,neighbourhood_group,latitude,longitude,price,criminality_index
0,0,Brooklyn,40.64749,-73.97237,149,1.000000
1,9,Brooklyn,40.68514,-73.95976,89,1.000000
2,9,Brooklyn,40.67989,-73.97798,80,1.000000
3,9,Brooklyn,40.68310,-73.95473,70,1.000000
4,0,Brooklyn,40.63702,-73.96327,150,1.000000
...,...,...,...,...,...,...
3490,9,Queens,40.75104,-73.81459,45,0.579804
3491,9,Queens,40.73710,-73.88086,50,0.579804
3492,9,Queens,40.69185,-73.86431,37,0.579804
3493,9,Queens,40.69183,-73.86523,34,0.579804


## Mapping the result

In [346]:
#to output to myself the df
#merged_airdf.to_excel("output.xlsx")  

### Due to excessive data we are going to filter to a 25% of our samples

In [341]:
new = merged_airdf.sample(frac = 0.4)
#new = pd.DataFrame(data=merged_airdf.head(1485))

In [342]:
new

,Cluster_Labels,neighbourhood_group,latitude,longitude,price,criminality_index
2949,9,Queens,40.68090,-73.78196,50,0.579804
1348,0,Manhattan,40.77550,-73.94750,130,0.747947
2899,0,Queens,40.72363,-73.80285,100,0.579804
1699,4,Manhattan,40.79099,-73.94500,400,0.747947
228,9,Brooklyn,40.68517,-73.93473,30,1.000000
...,...,...,...,...,...,...
1948,0,Manhattan,40.72546,-73.98396,179,0.747947
93,9,Brooklyn,40.69023,-73.94628,75,1.000000
542,4,Brooklyn,40.67476,-73.93016,550,1.000000
2757,9,Queens,40.71293,-73.84915,70,0.579804


In [343]:
# create map
latitude = 40.6643
longitude = -73.9385
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(new['latitude'], new['longitude'], new['neighbourhood_group'], new['Cluster_Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=3,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

In [344]:
map_clusters

# From best to worse: --> --> --> --> Purple -- Blue -- Green -- Orange -- Red

In [345]:
new.groupby('Cluster_Labels').size()

Cluster_Labels
0    416
1      2
2     23
3      3
4     91
5      3
6      1
7    183
8      5
9    671
dtype: int64